# 👁️ CoT Vision Factory (Llama 4 Scout Edition)

Bu notebook, Meta'nın en yeni **Llama 4 Scout (Native Multimodal)** modelini kullanarak makaleleri **hem okur hem de görür**.

**Farkı Nedir?**
- 🚫 Eski Yöntem: DePlot ile grafiği metne çevir -> LLM'e ver.
- ✅ Yeni Yöntem: Grafiğin **orijinal resmini** (piksel) -> LLM'e ver.

**Model:** `meta-llama/llama-4-scout-17b-16e-instruct`
Bu model, görsel ve metin verisini aynı anda işleyebilen (native multimodal) özel bir mimaridir.

## 1️⃣ Kurulum

In [5]:
from google.colab import drive
drive.mount('/content/drive')

!pip install groq pyyaml -q

import os
import sys
import glob
import time
import json
import base64
from google.colab import userdata

PROJECT_ROOT = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
sys.path.append(PROJECT_ROOT)

try:
    from analysis.cot_laboratory.core.assembler import CoTAssembler
    print("✅ CoT Vision Core Yüklendi!")
except ImportError as e:
    print(f"❌ HATA: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ CoT Vision Core Yüklendi!


In [6]:
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    print("✅ API Anahtarı Hazır.")
except:
    os.environ["GROQ_API_KEY"] = input("Groq API Key:")

✅ API Anahtarı Hazır.


## 2️⃣ Vision Helper (Base64 Encoder)

In [7]:
def encode_image(image_path):
    """Resmi Groq API için base64 formatına çevirir."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_paper_images(paper_folder, max_images=5):
    """Klasördeki en önemli 5 resmi bulur (Genelde boyutuna göre sıralarız)"""
    # Tüm resimleri bul
    types = ('*.png', '*.jpg', '*.jpeg')
    images = []
    for t in types:
        images.extend(glob.glob(os.path.join(paper_folder, t)))

    # Dosya boyutuna göre sırala (Büyük dosyalar genelde ana grafiklerdir, ikon değildir)
    images.sort(key=lambda x: os.path.getsize(x), reverse=True)

    # İlk N tanesini al
    selected_images = images[:max_images]

    encoded_images = []
    if selected_images:
        print(f"   📸 Seçilen Görseller ({len(selected_images)}): {[os.path.basename(i) for i in selected_images]}")
        for img in selected_images:
            encoded_images.append(encode_image(img))

    return encoded_images

## 3️⃣ Vision Factory (Run)

In [8]:
# Ayarlar
RECIPE_PATH = "analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml"
BATCH_START = 0
BATCH_END = 50
TEST_PAPER_ID = "O_ISAC_029"  # <-- Test için buraya ID yazın (Boş bırakırsanız tümünü yapar)

def run_vision_batch():
    assembler = CoTAssembler(PROJECT_ROOT)

    # Makaleleri Bul
    md_files = sorted(glob.glob(os.path.join(PROJECT_ROOT, "data/processed_markdowns/**/O_ISAC_*.md"), recursive=True))

    # Test Modu Kontrolü
    if TEST_PAPER_ID:
        target_papers = [p for p in md_files if TEST_PAPER_ID in p]
        print(f"🎯 TEST MODU AKTİF: Sadece {TEST_PAPER_ID} işlenecek.")
    else:
        target_papers = md_files[BATCH_START:BATCH_END]
        print(f"🚀 Batch Modu: {len(target_papers)} Makale Hedeflendi.")

    for idx, paper_path in enumerate(target_papers):
        paper_id = os.path.splitext(os.path.basename(paper_path))[0]
        print(f"\n[{BATCH_START + idx}] 👁️ Processing {paper_id}...")

        try:
            # 1. Metni Oku
            with open(paper_path, 'r', encoding='utf-8') as f:
                text_content = f.read()

            # 2. Resimleri Oku
            paper_dir = os.path.dirname(paper_path)
            images_b64 = get_paper_images(paper_dir, max_images=5)

            # 3. Multimodal Extraction
            result = assembler.run_extraction(
                RECIPE_PATH,
                text_content,
                paper_id=paper_id,
                image_data=images_b64 # <-- SİHİR BURADA
            )

            if result.get("status") == "success":
                print("   ✅ SUCCESS")
            else:
                print(f"   ❌ FAILED: {result.get('error_message')}")

            time.sleep(2) # Vision API rate limits might be stricter

        except Exception as e:
            print(f"   💥 CRITICAL: {e}")

run_vision_batch()

🎯 TEST MODU AKTİF: Sadece O_ISAC_029 işlenecek.

[0] 👁️ Processing O_ISAC_029...
   📸 Seçilen Görseller (5): ['_page_9_Figure_4.jpeg', '_page_10_Figure_2.jpeg', '_page_2_Figure_2.jpeg', '_page_5_Figure_2.jpeg', '_page_9_Figure_2.jpeg']
[INFO] Loading Recipe: analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml...
[INFO] 🖼️ Images detected! Switching model to: meta-llama/llama-4-scout-17b-16e-instruct
[INFO] Assembling System Prompt from Modules...
[INFO] Calling Groq API (Model: meta-llama/llama-4-scout-17b-16e-instruct)...

[DEBUG] RAW RESPONSE LEN: 7845
[DEBUG] RAW RESPONSE START: ```json
{
  "reasoning_trace": [
    {
      "key": "step_0_visual_inspection",
      "type": "string",
      "value": "The paper presents a THz photonic-based ISAC system for fiber-wireless networks,...
[DEBUG] CLEANED RESPONSE START: {
  "reasoning_trace": [
    {
      "key": "step_0_visual_inspection",
      "type": "string",
      "value": "The paper presents a THz photonic-based ISAC syste